In [1]:
import pandas as pd
import tqdm
import os
import re
import json 

In [2]:
CCLE_RNAseq_genes_tpm = pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/CCLE/CCLE_RNAseq_genes_tpm.csv')

In [3]:
gtex_autophogy_geneid=pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/gtex_autophogy_geneid.csv')

In [4]:
rna_cellline=pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/rna_celline.tsv', sep='\t')

In [5]:
Cell_lines_annotations=pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/CCLE/Cell_lines_annotations.csv')

## TPM

In [6]:
CCLE_RNAseq_genes_tpm.shape

(57820, 1021)

In [116]:
#add new genes
rna_tissue_gtex = pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/rna_tissue_gtex.tsv')

genes_id=rna_tissue_gtex.drop_duplicates('GeneName',keep='first')[['Gene','GeneName']]
new_id = genes_id[genes_id.GeneName. isin (['SQSTM1','CALCOCO2','NBR1','OPTN','TAX1BP1','NCOA4'])]

In [23]:
gtex_autophogy_geneid = pd.concat([new_id,gtex_autophogy_geneid],axis=0)
gtex_autophogy_geneid.to_csv('/Users/xinyutang/Documents/HPA2/Data/gtex_autophogy_geneid.csv')

In [24]:
# match geneid with autophagy id
CCLE_RNAseq_genes_tpm['gene_id']=CCLE_RNAseq_genes_tpm['gene_id'].apply(lambda x: x.split('.')[0])
ccle_autophogy = pd.merge(CCLE_RNAseq_genes_tpm,gtex_autophogy_geneid, left_on='gene_id', right_on='Gene')

In [25]:
#match cell line with columns
del ccle_autophogy['Gene']
del ccle_autophogy['transcript_ids']

ccle_autophogy_cell_line_id = ccle_autophogy.columns.tolist()[2:-3]
ccle_autophogy = pd.melt(ccle_autophogy,id_vars =['gene_id','GeneName'], 
                         value_vars =ccle_autophogy_cell_line_id,
                         var_name ='Cell_line', value_name ='TPM')


In [26]:
Cell_lines_annotations = Cell_lines_annotations[['CCLE_ID','Name','Site_Primary']]
Cell_lines_annotations['Lineage']=Cell_lines_annotations['CCLE_ID'].apply(lambda x: x.split('_')[1])
Cell_lines_annotations = Cell_lines_annotations.drop_duplicates('CCLE_ID',keep='first')

In [27]:
cell_line_dic = dict(zip(Cell_lines_annotations.CCLE_ID, Cell_lines_annotations.Name))
cell_line_dic2 = dict(zip(Cell_lines_annotations.CCLE_ID, Cell_lines_annotations.Lineage))

In [28]:
ccle_autophogy["Name"] = ccle_autophogy["Cell_line"].map(cell_line_dic)
ccle_autophogy["Site_Primary"] = ccle_autophogy["Cell_line"].map(cell_line_dic2)

In [37]:
ccle_autophogy=ccle_autophogy.rename({'CALCOCO2 (NDP52)':'CALCOCO2(NDP52)'},axis=1)


In [38]:
for i, j in ccle_autophogy.groupby('GeneName'):
    print(i)

ATG10
ATG101
ATG12
ATG13
ATG14
ATG16L1
ATG2A
ATG2B
ATG3
ATG4A
ATG4B
ATG4C
ATG4D
ATG5
ATG7
ATG9A
ATG9B
B1CC1(FIP200)
BECN1
BECN2
CALCOCO2 (NDP52)
GABARAP
GABARAPL1
GABARAPL2
MAP1LC3A
MAP1LC3B
MAP1LC3B2
MAP1LC3C
MCOLN1(TRPML1)
MCOLN1(TRPML3)
MCOLN2(TRPML2)
NBR1
OPTN
PIK3C3
PIK3CB
PIK3R4
RUBCN
SQSTM1(p62)
TAX1BP1
TFEB
ULK1
ULK2
ULK3
WDFY3
WIPI1
WIPI2


In [39]:
gtex_autophogy_geneid.to_csv('/Users/xinyutang/Documents/HPA2/Data/gtex_autophogy_geneid.csv')

In [52]:
ccle_autophogy['cellline']=ccle_autophogy['Cell_line'].apply(lambda x:x.split('_')[0])
ccle_autophogy.to_csv('/Users/xinyutang/Documents/HPA2/Data/CCLE/ccle_autophogy_tpm.csv')

## HPA cell-line

In [408]:
cellline_hpa = pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/CCLE/rna_celline.tsv',sep='\t')

In [410]:
cellline_hpa = cellline_hpa[['Gene','Gene name','Cell line','TPM']]

In [384]:
autophagyV8Url = np.load('/Users/xinyutang/Documents/HPA2/crawler/autophagyV8Url.npy')

In [122]:
new_html=[]
for html in autophagyV8Url:
    b = html.replace("v18", "v19")
    c =b.split('/')[:-1]
    d = "/".join(c).replace('tissue','cell')
    new_html.append(d)

In [23]:
url = np.load('/Users/xinyutang/Documents/HPA2/crawler/autophagyV9Url.npy',allow_pickle=True)
autophogy_gene = ['ATG10','ATG101','ATG12','ATG13','ATG14','ATG16L1',
              'ATG2A','ATG2B','ATG3','ATG4A','ATG4B','ATG4C','ATG4D','ATG5',
              'ATG7','ATG9A','ATG9B','BECN1','BECN2','CALCOCO2','GABARAP',
              'GABARAPL1','GABARAPL2','GABARAPL3',
              'MAP1LC3A','MAP1LC3B','MAP1LC3B2','MAP1LC3C',
              'MCOLN1','NCOA4','OPTN','PIK3C3','PIK3CB','PIK3R4',
              'RB1CC1','RUBCN','SQSTM1','TAX1BP1','TFEB','ULK1',
              'ULK2','ULK3','WDFY3','WIPI1','WIPI2','MCOLN2','MCOLN3']


In [26]:
autophagyurl=[]
for i in url:
    a = i.split('/')[3].split('-')[1]
    if a in autophogy_gene:
        autophagyurl.append(i)

In [92]:
autophagyurl = list(dict.fromkeys(autophagyurl))
np.save('/Users/xinyutang/Documents/HPA2/crawler/autophagyurl.npy',autophagyurl)


In [385]:
autophagyurl_v9 = np.load('/Users/xinyutang/Documents/HPA2/crawler/autophagyurl.npy')
autophagyurl_v9 = np.append(autophagyurl_v9, 'https://v19.proteinatlas.org/ENSG00000153898-MCOLN2/cell')
autophagyurl_v9 = np.append(autophagyurl_v9, 'https://v19.proteinatlas.org/ENSG00000055732-MCOLN3/cell')

In [43]:
autophagyurl_v9 = np.load('/Users/xinyutang/Documents/HPA2/crawler/autophagyurl.npy')

In [61]:
# Add URL

new_genes = ['ENSG00000106052-TAX1BP1',
'ENSG00000123240-OPTN',
'ENSG00000136436-CALCOCO2',
'ENSG00000161011-SQSTM1',
'ENSG00000188554-NBR1',
'ENSG00000266412-NCOA4']


for i in new_genes:
    first = 'https://v19.proteinatlas.org/'
    middle =i
    last = '/cell'
    print(middle)
    autophagyurl_v9 =np.append('https://v19.proteinatlas.org/ENSG00000188554-NBR1/cell',autophagyurl_v9)


ENSG00000106052-TAX1BP1
ENSG00000123240-OPTN
ENSG00000136436-CALCOCO2
ENSG00000161011-SQSTM1
ENSG00000188554-NBR1
ENSG00000266412-NCOA4


In [62]:
np.save('/Users/xinyutang/Documents/HPA2/crawler/autophagyurl.npy',autophagyurl_v9)

In [63]:
ccle_autophogy.columns

Index(['gene_id', 'GeneName', 'Cell_line', 'TPM', 'cellline', 'Lineage'], dtype='object')

In [70]:
path = '/Users/xinyutang/Documents/HPA2/Data/HPA-V9/'
l=[]
for r, d, f in os.walk(path):
    for file in f:
        file_path = path+file
        print(file_path)
        with open(file_path) as json_file:
         
            data = json.load(json_file)
            gene = list(data.keys())[0]
            
            for i in range(len(data[gene])):
                
                cellline = list(data[gene].keys())[i]
                organ = data[gene][cellline]
                df2 = pd.DataFrame({'GeneName': [gene], 'cellline': [cellline], 'Organ':[organ]})
                l.append(df2)
            

/Users/xinyutang/Documents/HPA2/Data/HPA-V9/ATG7.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/PIK3C3.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/TAX1BP1.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/MAP1LC3C.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/WIPI2.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/MCOLN3.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/MCOLN2.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/RB1CC1.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/ATG14.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/MAP1LC3B.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/ATG4D.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/NBR1.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/RUBCN.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/ATG9A.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/ATG2B.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/ATG13.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/WDFY3.json
/Users/xinyutang/Documents/HPA2/Data/HPA-V9/PIK3CB.json

In [71]:
final_df=pd.concat(l,axis=0)

In [72]:
final_df.columns

Index(['GeneName', 'cellline', 'Organ'], dtype='object')

In [78]:
rna_cellline=rna_cellline.rename({'Cell line':'cellline','Gene name':'GeneName'},axis=1)

In [76]:
del rna_cellline['Gene']
del rna_cellline['pTPM']
del rna_cellline['NX']

In [81]:
final_df.shape

(3008, 3)

In [83]:
cellline_hpa_autophagy = pd.merge(final_df,rna_cellline,on=['GeneName','cellline'],how='left')

In [85]:
    cellline_hpa_autophagy.GeneName=cellline_hpa_autophagy.GeneName.apply(lambda x: 'SQSTM1(p62)' if x=='SQSTM1' else\
                                                                   'CALCOCO2 (NDP52)' if x =='CALCOCO2'else x)


In [40]:
cellline_hpa_autophagy=pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/CCLE/cellline_hpa_autophagy.csv')

In [50]:
ccle_autophogy['cellline']=ccle_autophogy['Cell_line'].apply(lambda x:x.split('_')[0])

In [51]:
ccle_autophogy

,gene_id,GeneName,Cell_line,TPM,Name,Site_Primary,cellline
0,ENSG00000023287,B1CC1(FIP200),2313287_STOMACH,12.53,23132/87,STOMACH,2313287
1,ENSG00000034713,GABARAPL2,2313287_STOMACH,75.67,23132/87,STOMACH,2313287
2,ENSG00000051382,PIK3CB,2313287_STOMACH,41.99,23132/87,STOMACH,2313287
3,ENSG00000055732,MCOLN1(TRPML3),2313287_STOMACH,0.01,23132/87,STOMACH,2313287
4,ENSG00000057663,ATG5,2313287_STOMACH,32.39,23132/87,STOMACH,2313287
...,...,...,...,...,...,...,...
46777,ENSG00000196455,PIK3R4,ZR751_BREAST,11.03,ZR-75-1,BREAST,ZR751
46778,ENSG00000197548,ATG7,ZR751_BREAST,16.95,ZR-75-1,BREAST,ZR751
46779,ENSG00000197769,MAP1LC3C,ZR751_BREAST,0.00,ZR-75-1,BREAST,ZR751
46780,ENSG00000198925,ATG9A,ZR751_BREAST,18.03,ZR-75-1,BREAST,ZR751
